In [1]:
import json
import glob
import os
import random
import cv2 as cv2
import shutil
from PIL import Image
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pwd

/content/yolov5


In [ ]:
def convert_coco_json_to_yolov5_pytorch_format(json_file_path, txt_files_save_dir, images_dir, class_names):

# json_dir = '/Users/fe548/Downloads/custom_obj_det_data/'
# json_file_path = os.path.join(json_dir, 'all_images_data.json')
# print(json_file_path)
    if not os.path.exists(txt_files_save_dir):
        os.makedirs(txt_files_save_dir)
    
    json_file = open(json_file_path,)
    json_data = json.load(json_file)
#     txt_files_save_dir = '/Users/fe548/Downloads/custom_obj_det_data/labels/'
#     class_names = ['Apple', 'Orange', 'SweetPotato', 'Beetroot', 'Tomato1']
    images_data = json_data['images']
    images_data_dict = {dict_item['id']:dict_item for dict_item in images_data }
    annotations_data = json_data['annotations']
    person_count, car_count = 0, 0
    for index, annotations_dict_item in enumerate(annotations_data):
        annotation_id = annotations_dict_item['id']
        image_id = annotations_dict_item['image_id']
        bbox_values = annotations_dict_item['bbox']
        label_id = annotations_dict_item['category_id']
        label = [cat_dict_item['name'] for cat_dict_item in json_data['categories'] if cat_dict_item['id'] == label_id]
#         label = [cat_dict_item['id'] for cat_dict_item in json_data['categories'] if cat_dict_item['id'] == label_id]
#         class_id = label_id
        class_id = class_names.index(label[0])
        if class_id == 0:
            person_count = person_count + 1
        elif class_id == 1:
            car_count = car_count + 1
        image_specific_data = images_data_dict[image_id]
        height = image_specific_data['height']
        width = image_specific_data['width']
        image_path = images_dir + image_specific_data['file_name']
        image_name = image_path[image_path.rindex('/') + 1:]
        if height == 0 or width == 0:
            h, w, c = np.shape(cv2.imread(image_path))
    #     print(np.shape(cv2.imread(image_path)), image_path)
        h, w, c = np.shape(cv2.imread(image_path))
#         print(h, w, c)
        box = np.array(bbox_values, dtype=np.float64)
        try:
            box[:2] += box[2:] / 2  # xy top-left corner to center
            box[[0, 2]] /= width  # normalize x
            box[[1, 3]] /= height  # normalize y

            txt_file_name = image_name[0:image_name.rindex('.jpg')]
            line = class_id, * box


            txt_file_save_path = txt_files_save_dir + txt_file_name + '.txt'
#             print(txt_file_save_path)
            with open(txt_file_save_path, 'a+') as file:
                file.write(('%g ' * len(line)).rstrip() % line + '\n')
    
    #return person_count, car_count
#                 print('created txt file')
#             print(' ')
#             if index >= 3:
#                 break
        except Exception as e:
            print(e)
    return person_count, car_count

In [ ]:
json_file_path = '/content/drive/MyDrive/EV/trainval/annotations/bbox-annotations.json' 
txt_files_save_dir = '/content/drive/MyDrive/EV/yolo_data/txt_files/'
class_names = ['person', 'car']
images_dir = '/content/drive/MyDrive/EV/trainval/images/'
person_count, car_count = convert_coco_json_to_yolov5_pytorch_format(json_file_path, txt_files_save_dir, images_dir, class_names)

In [ ]:
print(person_count, car_count)

10800 5972


In [ ]:

def check_correctness_of_yolov5_labels():

    labels_txt_files_dir = '/content/drive/MyDrive/EV/yolo_data/txt_files'
    label_txt_files_names = os.listdir(labels_txt_files_dir)
    random_txt_files = random.sample(label_txt_files_names, 10)
    images_dir = '/content/drive/MyDrive/EV/trainval/images/'
    for index, txt_file_name in enumerate(random_txt_files):
        print(txt_file_name)
        image_name = txt_file_name[0:txt_file_name.rindex('.txt')]+ '.jpg'
        img = cv2.imread(images_dir + image_name)

        txt_file = open(labels_txt_files_dir + txt_file_name, 'r')
        Lines = txt_file.readlines()
        h, w, c = np.shape(cv2.imread(images_dir + image_name))
    #     count = 0
        # Strips the newline character
        for line in Lines:
            class_bbox_values = line.split()
            class_bbox_values = list(map(lambda x:float(x), class_bbox_values))

            class_id = class_bbox_values[0]
            bbox_values = class_bbox_values[1:]

            x_center = float(bbox_values[0])
            y_center = float(bbox_values[1])
            width = float(bbox_values[2])
            height = float(bbox_values[3])

            int_x_center = int(w*x_center)
            int_y_center = int(h*y_center)
            int_width = int(w*width)
            int_height = int(h*height)

            min_x = int_x_center-int_width/2
            min_y = int_y_center-int_height/2
            width = int_width
            height = int_height

            start_point = (int(min_x), int(min_y))
            end_point = (int(min_x +width), int(min_y + height))

            thickness = 2
            color = (255, 0, 0)
            cv2.rectangle(img, start_point, end_point, color, thickness)
            Image.fromarray(img).show()
    #         print(class_bbox_values)
    #         print(bbox_values)
            print(' ')
        if index >= 3:
            break
    
    

In [3]:

def check_if_directory_is_there(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def divide_data_into_train_and_valid(imgs_dir, label_txt_files_dir, dataset_dir):
    
    
    train_imgs_dir = dataset_dir + 'train/images/'
    train_labels_dir = dataset_dir + 'train/labels/'
    
    valid_imgs_dir = dataset_dir + 'valid/images/'
    valid_labels_dir = dataset_dir + 'valid/labels/'
    check_if_directory_is_there(dataset_dir)
    check_if_directory_is_there(dataset_dir + 'train')
    check_if_directory_is_there(dataset_dir + 'valid')
    check_if_directory_is_there(train_imgs_dir)
    check_if_directory_is_there(train_labels_dir)
    check_if_directory_is_there(valid_imgs_dir)
    check_if_directory_is_there(valid_labels_dir)
    
#     check_if_directory_is_there(valid_data_dir)
    
    imgs_names = os.listdir(imgs_dir)
    txt_files_names = os.listdir(label_txt_files_dir)
    num_imgs = len(imgs_names)
    print(num_imgs, len(txt_files_names))
    #assert num_imgs == len(txt_files_names)
    
    valid_imgs_names = random.sample(imgs_names, 50)
    
    for img_name in imgs_names:
    
        txt_file_name = img_name[0:img_name.rindex('.jpg')] + '.txt'

        if txt_file_name in txt_files_names:
            
            if img_name in valid_imgs_names:
                shutil.copy(imgs_dir + img_name, valid_imgs_dir + img_name)
                shutil.copy(label_txt_files_dir + txt_file_name, valid_labels_dir + txt_file_name)
            else:
                shutil.copy(imgs_dir + img_name, train_imgs_dir + img_name)
                shutil.copy(label_txt_files_dir + txt_file_name, train_labels_dir + txt_file_name)

            
imgs_dir = '/content/drive/MyDrive/EV/yolo_data/images/'
label_txt_files_dir = '/content/drive/MyDrive/EV/yolo_data/txt_files/'
dataset_dir = '/content/drive/MyDrive/EV/yolo_data/dataset2/'
divide_data_into_train_and_valid(imgs_dir, label_txt_files_dir, dataset_dir)
    

4252 4181


In [ ]:
! ls -1 /content/drive/MyDrive/EV/yolo_data/dataset/train/labels | wc -l

! ls -1 /content/drive/MyDrive/EV/yolo_data/dataset/valid/labels | wc -l

2210


In [ ]:
train_imgs = '/content/drive/MyDrive/EV/yolo_data/dataset/train/images/'
valid_imgs= '/content/drive/MyDrive/EV/yolo_data/dataset/valid/images/'

train_imgs = os.listdir(train_imgs)
valid_imgs = os.listdir(valid_imgs)